In [23]:
%pip install spark-nlp

    100% |████████████████████████████████| 153kB 3.4MB/s ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [93]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from operator import add
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import sparknlp
from sparknlp.annotator import LemmatizerModel
from sparknlp.pretrained import PretrainedPipeline
spark = sparknlp.start()

In [94]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer

In [145]:
import pandas as pd

In [96]:
#spark = SparkSession\
 #   .builder \
  #  .appName("PythonWordCount") \
   # .appName("Spark NLP")\
    #.master("local[*]")\
#    .config("spark.driver.memory","16G")\
 #   .config("spark.driver.maxResultSize", "0") \
  #  .config("spark.kryoserializer.buffer.max", "2000M")\
   # .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.0.1")\
    #.getOrCreate()
SparkSession.builder.config("spark.jars", "hdfs://somepath/sparknlp.jar")    

In [97]:
from sparknlp.base import Finisher, DocumentAssembler
from sparknlp.annotator import (Tokenizer, Normalizer,
                                LemmatizerModel, StopWordsCleaner)
from pyspark.ml import Pipeline

In [98]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import udf, concat, col, lit
import re
from pyspark.sql.types import IntegerType

In [99]:
hacker = spark.read.format('csv').options(header='true', inferSchema='true') \
    .load('/spring2021/project1/hacker_news_sample.csv')

In [100]:
hacker = hacker.na.drop(subset=["text"])

In [101]:
hacker.show()

+---------------+----+--------------------+----+---------------+-----+----------+-------+--------+--------+-----------+-------+-------+-------------------+
|          title| url|                text|dead|             by|score|      time|   type|      id|  parent|descendants|ranking|deleted|          timestamp|
+---------------+----+--------------------+----+---------------+-----+----------+-------+--------+--------+-----------+-------+-------+-------------------+
|           null|null|&gt;<i>which lead...|null|        coldtea| null|1390843873|comment| 7131680| 7127578|       null|   null|   null|2014-01-27 12:31:13|
|           null|null|I would like to p...|null|         etanol| null|1319395600|comment| 3146879| 3145330|       null|   null|   null|2011-10-23 14:46:40|
|           null|null|<i>Our msbuild im...|null|      Locke1689| null|1407881590|comment| 8170491| 8170071|       null|   null|   null|2014-08-12 18:13:10|
|           null|null|No matter how awf...|null|    miloshadzic|

In [102]:
hacker.withColumn('word', f.explode(f.split(f.col('text'), ' '))) \
    .groupBy('word') \
    .count() \
    .sort('count', ascending=False) \
    .show()

+----+------+
|word| count|
+----+------+
| the|346807|
|  to|260578|
|   a|224952|
|  of|186141|
| and|170948|
|  is|145182|
|that|132135|
|   I|121470|
|  in|113588|
| you| 89622|
| for| 89295|
|  it| 87574|
|    | 64798|
|  be| 63524|
|  on| 61459|
| are| 60230|
|with| 59479|
|have| 56174|
| not| 55318|
| but| 49318|
+----+------+
only showing top 20 rows



In [103]:
tokenizer = Tokenizer(inputCol="text", outputCol="words_token")
tokenized = tokenizer.transform(hacker).select('text','words_token')
regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words_token")
regexTokenized = regexTokenizer.transform(hacker).select('words_token')
regexTokenized.show()

+--------------------+
|         words_token|
+--------------------+
|[&gt;<i>which, le...|
|[i, would, like, ...|
|[<i>our, msbuild,...|
|[no, matter, how,...|
|[the, existence, ...|
|[the, actual, int...|
|[i, want, to, kno...|
|[french, is, supr...|
|[i, actually, wen...|
|[first, impressio...|
|[it&#x27;s, not, ...|
|[thanks., i, choo...|
|[i, used, sybase,...|
|[&gt;they&#x27;ve...|
|[if, there's, a, ...|
|[(i, submitted, t...|
|["actually, the, ...|
|[&#62;, if, the, ...|
|[looks, like, the...|
|[definitely, has,...|
+--------------------+
only showing top 20 rows



In [115]:
def splite(value):
    punc = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    for ch in punc:
        value = value.replace(ch, ' ')
    value = value.replace(' ','')
    return value
import string

def split(value):
    for i in range (len(string.punctuation)):
        value = value.replace(string.punctuation[i], '')
    return value

nopunc=splite(data_clean)

In [118]:
remover = StopWordsRemover(inputCol='words_token', outputCol='words_clean')
data_clean = remover.transform(regexTokenized).select( 'words_clean')
data_clean.show()

+--------------------+
|         words_clean|
+--------------------+
|[&gt;<i>which, le...|
|[like, point, cou...|
|[<i>our, msbuild,...|
|[matter, awful, i...|
|[existence, way, ...|
|[actual, internet...|
|[want, know, ants...|
|[french, supremel...|
|[actually, went, ...|
|[first, impressio...|
|[it&#x27;s, exact...|
|[thanks., choose,...|
|[used, sybase, as...|
|[&gt;they&#x27;ve...|
|[high, chance, go...|
|[(i, submitted, d...|
|["actually, defin...|
|[&#62;, applicati...|
|[looks, like, 10k...|
|[definitely, slan...|
+--------------------+
only showing top 20 rows



In [106]:
result = data_clean.withColumn('word', f.explode(f.col('words_clean'))) \
    .groupBy('word') \
    .count().sort('count', ascending=False) \
    .show()

+----------+-----+
|      word|count|
+----------+-----+
|      like|31676|
|    people|28749|
|       one|22442|
|     think|21404|
| it&#x27;s|20890|
|       get|20059|
|       use|15903|
|      even|15826|
|         -|15537|
|      much|15440|
|don&#x27;t|15340|
|    really|14927|
|      make|14311|
|      also|14174|
|      time|14140|
|      good|13925|
|        <a|13485|
|      know|12729|
|      work|12603|
|       way|12292|
+----------+-----+
only showing top 20 rows



In [119]:
hackernew=nopunc.toPandas()

Exception ignored in: <bound method JavaWrapper.__del__ of Normalizer_1e25809075a8>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/pyspark/ml/wrapper.py", line 40, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'Normalizer' object has no attribute '_java_obj'


In [120]:
hackernew.head()

,words_clean
0,"[&gt;<i>which, leads, say, using, c, x?</i><p>..."
1,"[like, point, counter-examples:<p>«<i>think, j..."
2,"[<i>our, msbuild, implementation, build, proje..."
3,"[matter, awful, iphoto, still, better, almost,..."
4,"[existence, way, shard, searches, make, scalin..."


In [194]:
def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = re.sub("[^a-zA-Z]",  # Search for all non-letters
                          " ",          # Replace all non-letters with spaces
                          str(text))
    text = re.sub('\[.*?\]', '', str(text))
    text = re.sub('[%s]' % re.escape(string.punctuation), '', str(text))
    text = re.sub('\w*\d\w*', '', str(text))
    return text

round1 = lambda x: clean_text_round1(x)

In [195]:
data_clean = pd.DataFrame(hackernew.words_clean.apply(round1))

data_clean

,words_clean
0,gt i which leads say using c ...
1,like point counter examples p i thi...
2,i our msbuild implementation build...
3,matter awful iphoto still better...
4,existence way shard searches mak...
...,...
162138,oh wow didn x t notice settin...
162139,using weeks think rocks great...
162140,matters p reads hn works ...
162141,it x s giving up it x s e...


In [196]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text
round2 = lambda x: clean_text_round2(x)

In [198]:
data_clean2 = pd.DataFrame(data_clean.words_clean.apply(round2))
data_clean2

,words_clean
0,gt i which leads say using c ...
1,like point counter examples p i thi...
2,i our msbuild implementation build...
3,matter awful iphoto still better...
4,existence way shard searches mak...
...,...
162138,oh wow didn x t notice settin...
162139,using weeks think rocks great...
162140,matters p reads hn works ...
162141,it x s giving up it x s e...


In [204]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_clean2.words_clean)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm

,aa,aaa,aaaa,aaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa,aaaaaaaaaai,aaaaaaaaarw,aaaaaaaaau,aaaaaaaaaum,aaaaaaaaav,...,zyxo,zz,zzbrs,zziar,zzk,zzleeper,zztthyho,zzyryya,zzz,zzzcpan
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162138,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
162139,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
162140,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
162141,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [206]:
data = data_dtm.transpose()

data.head()

,0,1,2,3,4,5,6,7,8,9,...,162133,162134,162135,162136,162137,162138,162139,162140,162141,162142
aa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaaa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaaaaaaaa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [121]:
from collections import Counter

In [209]:
top_dict = {}
for i in data.columns:
    top = data[i].sort_values(ascending=False).head(30)
    top_dict[i]= list(zip(top.values,top.index))
    
top_dict

KeyboardInterrupt: 

In [140]:
words = []
for i in data.columns:
    top = [word for (word, count) in top_dict[i]]
    for t in top:
        words.append(t)
        
words

[154605,
 108640,
 146599,
 138804,
 41237,
 141588,
 13209,
 41161,
 78650,
 73537,
 70011,
 154714,
 143259,
 127764,
 25625,
 47175,
 30971,
 83446,
 27625,
 35337,
 42161,
 160820,
 144652,
 40159,
 46137,
 38313,
 106624,
 10134,
 105929,
 130265]

In [210]:
Counter(data).most_common()

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 1),
 (21, 1),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 1),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 1),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 1),
 (39, 1),
 (40, 1),
 (41, 1),
 (42, 1),
 (43, 1),
 (44, 1),
 (45, 1),
 (46, 1),
 (47, 1),
 (48, 1),
 (49, 1),
 (50, 1),
 (51, 1),
 (52, 1),
 (53, 1),
 (54, 1),
 (55, 1),
 (56, 1),
 (57, 1),
 (58, 1),
 (59, 1),
 (60, 1),
 (61, 1),
 (62, 1),
 (63, 1),
 (64, 1),
 (65, 1),
 (66, 1),
 (67, 1),
 (68, 1),
 (69, 1),
 (70, 1),
 (71, 1),
 (72, 1),
 (73, 1),
 (74, 1),
 (75, 1),
 (76, 1),
 (77, 1),
 (78, 1),
 (79, 1),
 (80, 1),
 (81, 1),
 (82, 1),
 (83, 1),
 (84, 1),
 (85, 1),
 (86, 1),
 (87, 1),
 (88, 1),
 (89, 1),
 (90, 1),
 (91, 1),
 (92, 1),
 (93, 1),
 (94, 1),
 (95, 1),
 (96, 1),
 (97, 1),
 (98, 1),
 (99, 1),
 (100, 1),